# Development API call to Database
* Author: Stefan Roland Schwingenschlögl
* email: stefan.roland.schwingenschloegl@gmail.com
* github: https://github.com/stefan-schwingenschloegl/
---
This notebooks is intended to develop the pipeline to call the API and ingest the information into the MS SQL Server. This code is then shifted to kedro nodes and pipeline.

In [1]:
# import libraries
import pandas as pd
import requests

from typing import Dict, List

# Overall pipeline method

<ol>
    <li>Call API and validate response</li>
    <li>Save as pandas pq file</li>
</ol>

In [2]:
def API_database_ingestion_pipeline(api_response, 
                                    api_error_dict,
                                    db_column_names,
                                    direction_to_consider
                                   ):
    """
    Main method to call all transformation steps of the data pipeline
    """

    # call API and validate response
    api_response = call_API(api_response, api_error_dict)

    # get next depart and convert to relational db format
    relational_data = get_next_depart_and_convert_format(db_column_names, api_response)

    # prefilter to right directions
    output_data = prefilter_for_direction(relational_data, direction_to_consider)

    return output_data

## Call API and validate response

In [3]:
def call_API(api_response: requests.models.Response,
             api_error_dict: Dict) -> Dict:
    """
    Method to call API defined by URL in data catalog. Check if Call was sucessfull and if not which error it is.

    :param api_response requests.models.Response: API response 
    :param api_error_dict Dict: Dictionary with all error codes from documentation
    :return: if sucessfull call -> API response in dict format
    :raises ValueError: If response of API is not 200 -> Raise error according to return code.
    """    

    # Check if status of query was successfull
    if api_response.status_code == 200:
        print(f"API call for successful!")
        return api_response.json()

    elif api_response.status_code in api_error_dict.keys():
        raise ValueError(f"API call not successful due to Wiener Linien internal defined error!\
        Reason {api_response.status_code}: {api_error_dict[api_response.status_code]}") 

    else:
        raise ValueError(f"API call not successful and status code not defined by to Wiener Linien!")

# Prefilter API Response to get the next depature per stopID and transform into db staging table schema

In [4]:
def get_next_depart_and_convert_format(db_column_names: List[str],
                                       api_response: Dict) -> pd.DataFrame:
    """
    Iterate through all stops and get the next line which is going to depart from each stop

    :params df_schema List[str]: List of strings with the column names
    :params api_response Dict: json response from the API
    :returns: dataframe with every next depature per stop
    """
    
    
    return_df = pd.DataFrame()
    timestamp = pd.json_normalize(api_response)['message.serverTime'].values[0]

    for num, stop in enumerate(api_response['data']['monitors']):
        # get stop attributes
        stop_normalized = pd.json_normalize(api_response['data']['monitors'][num])
        stop_name = stop_normalized['locationStop.properties.title'].values[0]
        rbl = stop_normalized['locationStop.properties.attributes.rbl'].values[0]

        # get lines attributes
        lines_normalized = pd.json_normalize(stop_normalized['lines'].values[0])
        lineID = lines_normalized['lineId'].values[0]
        directionID = lines_normalized['richtungsId'].values[0]

        # get realtime information
        depature_information =pd.json_normalize(lines_normalized['departures.departure'].values[0])
        time_planned = depature_information['departureTime.timePlanned'].values[0]
        time_real = depature_information['departureTime.timePlanned'].values[0]

        single_entry = pd.DataFrame(columns = db_column_names,
                                    data = [[timestamp, 
                                             stop_name, 
                                             rbl, 
                                             lineID, 
                                             directionID, 
                                             time_planned, 
                                             time_real]])
        return_df = pd.concat([return_df, single_entry])

        # convert datatypes
        for col in ['API_ServerTime', 'TimePlanned', 'TimeReal']:
            return_df[col] = pd.to_datetime(return_df[col])
        
    return return_df

# Prefilter for line direction to consider

In [5]:
def prefilter_for_direction(api_response_df: pd.DataFrame,
                            directions_to_consider: List[str]) -> pd.DataFrame:
    """
    Prefilter Dataframe to only to consider wanted directions

    :param api_response_df pd.DataFrame: API response in pandas Dataframe Format
    :param directions_to_consider List[str]: List of strings for all Directions we want to consider
    :returns pd.DataFrame: Prefiltered pandas Dataframe with API response
    """

    api_response_df = api_response_df.loc[api_response_df['Direction'].astype(str).isin(directions_to_consider)]
    return api_response_df

# Run pipeline

In [6]:
# mockup data inputs to pipeline
mockup_api_response = catalog.load("API_Call_13A")
mockup_api_error_dict = catalog.load("params:api_error_dict")
mockup_column_names = catalog.load("params:dataframe_schema_for_db")
mockup_direction = catalog.load("params:line_directions_to_consider")

# Run pipeline
data_output = API_database_ingestion_pipeline(mockup_api_response, 
                                              mockup_api_error_dict,
                                              mockup_column_names,
                                              mockup_direction)

[01/06/24 22:18:10] INFO     Loading data from API_Call_13A (APIDataset)...                     ]8;id=909363;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=390258;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py#482\482]8;;\

                    INFO     Loading data from params:api_error_dict (MemoryDataset)...         ]8;id=141315;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=583787;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py#482\482]8;;\

                    INFO     Loading data from params:dataframe_schema_for_db                   ]8;id=274585;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=130622;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py#482\482]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:line_directions_to_consider               ]8;id=424124;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=343652;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py#482\482]8;;\
                             (MemoryDataset)...                                                                    

API call for successful!


# Write to database

The following Code was used to generate the stage table in MS SQL Server:
```
Use WienerLinienDB

IF NOT EXISTS(SELECT * FROM information_schema.tables
                 WHERE table_name='stage_api_response' AND table_schema='dbo') 
                 CREATE TABLE stage_api_response (
                        [API_ServerTime] datetime2,
                        [StopName] varchar(100),
						[rbl] int,
						[LineID] int,
						[Direction] int,
						[TimePlanned] datetime2,
						[TimeReal] datetime2)

SELECT * FROM stage_api_response;

```